In [ ]:
import random
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from collections import Counter
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

%matplotlib inline

# K-Means

## Algorithm

In this section, you need to implement your own K Means algorithm. Please follow the instructions. 

In [ ]:
def k_means(X, k, max_iter=100):
    
    """
    Args:
    - X : feature matrix, np.array
    - k : number of clusters
    - max_iter : maximum iteratations

    Returns:
    - centers: k centers
    - labels: labels assigned for each data point
    """
    
    # randomly choose k points as initialized centers
    # convert each center to tuple type
    centers_ind = np.random.choice(range(X.shape[0]), size=k)
    centers = X[centers_ind]
    
    # loop max_iter
    for i in range(max_iter):
        
        
        ################ Step 1: Assignment ################
        # compute distances between X and centers, using pairwise distance
        distances = euclidean_distances(X, centers)
        
        # get labels of each data point
        labels = np.argmin(distances, axis=1)
        
        
        ################ Step 2: Compute New Centers ################
        # get new centers
        new_centers = centers.copy()
        for i in range(len(centers)):
            
            # find data points with center i
            data = X[np.where(labels == i)]
            
            # update each center by averaging data
            new_centers[i] = np.mean(data, axis=0)
        
        
        ################ Step 3: Check Convergence ################
        # compute distance between new centers and old centers, using pairwise distance
        mat = euclidean_distances(new_centers, centers)
        
        # sum up diagonal values
        error = np.sum([mat[i,i] for i in range(len(mat))])
        
        # if error is less than 1e-6, break
        if error < 1e-6:
            break
        
        
        ################ Step 4: Update Centers ################
        centers = new_centers
    
    return centers, labels

## Road Network

In [ ]:
# read data
road_network = []

with open("../data/spatial_network.txt", 'r') as f:
    for line in f.readlines():
        
        # split by comma, only get LONGITUDE and LATITUDE, then convert to float
        record = list(map(float, line.strip().split(',')[1:3]))
        
        # append to road_network
        road_network.append(record)


In [ ]:
X = np.array(road_network)

# set seed
np.random.seed(42)

# YOUR OWN MODEL
centers, labels = k_means(X, k=3, max_iter=100)

# SKLEARN MODEL
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42).fit(X)
centers_sklearn, labels_sklearn = kmeans.cluster_centers_, kmeans.labels_

### Test

In [ ]:
# test for the data
assert X.shape == (434874, 2)
assert round(X[666,1],4) == 56.6474

# test for k means algorithm
assert round(centers[0][0],4) == 8.7975
assert round(centers[0][1],4) == 56.8675
assert labels[111111] == 2
assert labels[222222] == 1
assert labels[333333] == 1

## Visualization

In [ ]:
LABELS = [labels, labels_sklearn]
TITLES = ["YOUR OWN MODEL", "SCIKIT-LEARN"]

# it might take seconds to run

for i in range(2):
    plt.figure(figsize=(12,12))
    plt.subplot(2,1,i+1)
    plt.scatter(X[:, 0], X[:, 1], c=LABELS[i], cmap='viridis')
    plt.xlabel("LONGITUDE")
    plt.ylabel("LATITUDE")
    plt.title("Clustering by %s"%TITLES[i])
    plt.show()

## Elbow Method

In [ ]:
def sse(centers, labels):
    """Sum squared euclidean distance of all points to their cluster center"""
    
    res = 0
    # loop each label
    for label in set(labels):
        
        # get center
        c = centers[label].reshape(1,-1)
        
        # get data points
        pts = X[np.where(labels==label)]
        
        # compute sum of square error
        ss = euclidean_distances(c, pts).sum()
        
        # add to result
        res += ss
    
    return res


def plot_k_sse(X, min_k, max_k):
    """
    Args:
    - X : feature matrix, 
    - min_k, max_k : smallest and largest k to plot sse for
    """
    k_values = range(min_k, max_k+1)
    sse_values = []
    for k in k_values:
        # train k means model
        centers, labels = k_means(X, k=k)
        # compute sse
        SSE = sse(centers, labels)
        print (k, ": ", int(SSE))
        # append to list
        sse_values.append(SSE)
    # plot
    plt.figure(figsize=(10,6))
    plt.plot(k_values, sse_values)
    plt.xlabel('k')
    plt.ylabel('sum squared error')


In [ ]:
# set seed
np.random.seed(42)

plot_k_sse(X, 2, 10)

## Silhouette Coefficient

In [ ]:
def plot_k_silhouette(X, min_k, max_k):
    """Plots sse for values of k between min_k and max_k

    Args:
    - X : feature matrix
    - min_k, max_k : smallest and largest k to plot sse for
    """
    k_values = range(min_k, max_k+1)
    silhouette_scores = []
    
    for k in k_values:
        # train model
        centers, labels = k_means(X, k=k)
        # compute score, with sampling
        score = silhouette_score(X, labels, sample_size = 10000)
        print (k, ": ", score)
        silhouette_scores.append(score)

    # plot
    plt.figure(figsize=(10,6))
    plt.plot(k_values, silhouette_scores)
    plt.xlabel('k')
    plt.ylabel('silhouette score')

In [ ]:
# set seed
np.random.seed(42)

plot_k_silhouette(X, 2, 10)

## Gap Statistic

https://github.com/milesgranger/gap_statistic/blob/master/gap_statistic/optimalK.py

In [ ]:
from gap_statistic import OptimalK

optimalK = OptimalK(parallel_backend=None)
optimalK(X, cluster_array=range(2,11))

# Hierarchical Clustering

Now we are going to leverage [Scipy](http://www.scipy.org/) to perform [hierarchical clustering](http://en.wikipedia.org/wiki/Hierarchical_clustering) using `New York Times articles`.

1. Hierarchical clustering is more computationally intensive than Kmeans.  Also it is hard to visualize the results of a hierarchical clustering if you have too much data (since it represents its clusters as a tree).   

2. The first step to using `scipy's` Hierarchical clustering is to first find out how similar our vectors are to one another. To do this we use the `euclidean distances` to compute a similarity matrix of our data (pairwise distances). 

3. Now that we have a square similarity matrix we can start to cluster!  Pass this matrix into scipy's `linkage` [function](http://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) to compute our hierarchical clusters.

4. We in theory have all the information about our clusters but it is basically impossible to interpret in a sensible manner.  Thankfully scipy also has a function to visualize this madness.  Using scipy's `dendrogram` [function](http://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) plot the linkages as a hierachical tree.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.hierarchy import linkage, dendrogram

articles_df = pd.read_csv("../data/articles.csv")

In [ ]:
# TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(articles_df['content'])


# Distance Matrix
dist_mat = euclidean_distances(X.todense(), X.todense())


# Linkage
link = linkage(dist_mat, method='complete')

In [ ]:
# dendrogram plot
plt.figure(figsize=(16,8))
dendro = dendrogram(link, color_threshold=2.01, leaf_font_size=9,
                    labels=small_df['section_name'].values)
plt.ylim(1,2.2)
plt.show()

Alternative Hierarchical Clustering in Scikit-Learn.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html

# DBSCAN

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

In [ ]:
from sklearn.cluster import DBSCAN

X = np.array(road_network)
dbscan = DBSCAN(eps=0.01, min_samples=2).fit(X)
labels_dbscan = dbscan.labels_


In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(X[:, 0], X[:, 1], c=labels_dbscan, cmap='viridis')
plt.xlabel("LONGITUDE")
plt.ylabel("LATITUDE")
plt.title("Clustering by DBSCAN")
plt.show()